# Medical Chatbot Development Notebook
## Using LangChain 1.0 + Groq + Pinecone

In [ ]:
print("ok")

In [ ]:
%pwd

In [ ]:
import os
os.chdir('../')

In [ ]:
%pwd

In [ ]:
# Modern imports - Compatible with LangChain 1.0+
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
def load_pdf_files(data):
    """Load all PDF files from directory"""
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [ ]:
# Load PDFs from data folder
extracted_docs = load_pdf_files('data')
print(f"Loaded {len(extracted_docs)} documents")

In [ ]:
from typing import List
from langchain_core.documents import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """Keep only essential metadata"""
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get('source')
        minimal_docs.append(
            Document(page_content=doc.page_content, metadata={'source': src})
        )
    return minimal_docs

In [ ]:
minimal_docs = filter_to_minimal_docs(extracted_docs)
print(f"Filtered to {len(minimal_docs)} minimal documents")

In [ ]:
def text_split(minimal_docs):
    """Split documents into chunks"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [ ]:
texts_chunk = text_split(minimal_docs)
print(f"Split into {len(texts_chunk)} chunks")

In [ ]:
# Initialize embeddings model
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """Initialize HuggingFace embeddings"""
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings

embedding = download_embeddings()
print("✅ Embeddings model loaded")

In [ ]:
# Test embeddings
vectors = embedding.embed_query("Hello world")
print(f"Vector dimension: {len(vectors)}")

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
print("✅ Environment variables loaded")

In [ ]:
# Get API keys from environment (SECURE WAY)
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not PINECONE_API_KEY:
    print("❌ PINECONE_API_KEY not found")
else:
    print("✅ Pinecone API key loaded")
    
if not GROQ_API_KEY:
    print("❌ GROQ_API_KEY not found")
else:
    print("✅ Groq API key loaded")

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
print("✅ Connected to Pinecone")

In [ ]:
from pinecone import ServerlessSpec

index_name = "medical-chatbot"

if not pc.has_index(index_name):
    print(f"Creating index '{index_name}'...")
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print("✅ Index created")
else:
    print(f"✅ Index '{index_name}' already exists")

index = pc.Index(index_name)

## Check if Documents Already Exist

In [ ]:
from langchain_pinecone import PineconeVectorStore

# Check if vectors already exist
index_stats = pc.Index(index_name).describe_index_stats()
current_count = index_stats.get('total_vector_count', 0)

print(f"Current vectors in index: {current_count}")

if current_count == 0:
    # First time: Create vectors
    print("Creating vector store (first time)...")
    docsearch = PineconeVectorStore.from_documents(
        documents=texts_chunk,
        embedding=embedding,
        index_name=index_name
    )
    print(f"✅ Created {len(texts_chunk)} vectors")
else:
    # Already has data: Just load it
    print("Vector store exists. Loading...")
    docsearch = PineconeVectorStore.from_existing_index(
        embedding=embedding,
        index_name=index_name
    )
    print(f"✅ Loaded existing store with {current_count} vectors")

In [ ]:
# OR load existing vector store
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    embedding=embedding,
    index_name=index_name
)
print("✅ Loaded existing vector store")

In [ ]:
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)
print("✅ Retriever created")

In [ ]:
# Test retrieval
retrieved_docs = retriever.invoke("What is Acne?")
print(f"Retrieved {len(retrieved_docs)} documents")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\nDoc {i}: {doc.page_content[:200]}...")

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser




chatModel = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    groq_api_key=GROQ_API_KEY,
    temperature=0.3,      # Balanced creativity vs accuracy
    max_tokens=1024       # Allow longer, detailed responses
)

# Better retriever - get more context
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # Retrieve 5 documents instead of 3
)

print("✅ Optimized model initialized")
print("   Model: Llama 3.3 70B")
print("   Temperature: 0.3")
print("   Max tokens: 1024")
print("   Retrieval: Top 5 documents")

In [ ]:




system_prompt = """You are an expert Medical Assistant with comprehensive knowledge of medical conditions, treatments, and healthcare.

Your role is to provide detailed, accurate, and helpful answers based on the medical literature provided in the context.

Guidelines for your responses:
1. **Be Comprehensive**: Provide thorough explanations covering all relevant aspects
2. **Be Structured**: Organize information logically (definition, causes, symptoms, treatment, etc.)
3. **Be Clear**: Explain medical terms in understandable language
4. **Be Accurate**: Only use information from the provided context
5. **Be Helpful**: Anticipate follow-up questions and address them
6. **Be Honest**: If information is missing from context, clearly state it

Context from medical literature:
{context}

Provide a detailed, well-structured answer to the following question:"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

print("✅ Enhanced medical prompt created")


In [ ]:



def format_docs(docs):
    """Format documents with clear structure"""
    formatted = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', 'Unknown')
        content = doc.page_content.strip()
        formatted.append(f"[Document {i} - {source}]:\n{content}")
    return "\n\n" + "="*60 + "\n\n".join(formatted)

print("✅ Document formatter ready")

In [ ]:

rag_chain = (
    {
        "context": retriever | format_docs,
        "input": RunnablePassthrough()
    }
    | prompt
    | chatModel
    | StrOutputParser()
)

print("✅ Optimized RAG chain ready!")
print("\nImprovements:")
print("  • 5 source documents (more context)")
print("  • Enhanced prompt for deeper answers")
print("  • Better document formatting")
print("  • Llama 3.3 70B for best quality")


In [ ]:
import time
from typing import Dict

def ask_medical_question(
    question: str,
    show_sources: bool = True,
    show_timing: bool = True,
    show_retrieved_docs: bool = False
) -> Dict:
    """
    Ask a comprehensive medical question
    
    Args:
        question: Your medical question
        show_sources: Display sources used (default: True)
        show_timing: Show response time (default: True)
        show_retrieved_docs: Show full retrieved documents (default: False)
    
    Returns:
        Dictionary with answer and metadata
    """
    
    start_time = time.time()
    
    # Header
    print("\n" + "="*80)
    print(f"📋 QUESTION: {question}")
    print("="*80 + "\n")
    
    # Get answer
    print("🤖 Generating comprehensive answer...\n")
    response = rag_chain.invoke(question)
    
    elapsed_time = time.time() - start_time
    
    # Display answer with nice formatting
    print("💡 DETAILED ANSWER:")
    print("-" * 80)
    print(response)
    print("-" * 80)
    
    # Show sources
    if show_sources:
        retrieved_docs = retriever.invoke(question)
        print(f"\n📚 SOURCES CONSULTED ({len(retrieved_docs)} documents):")
        for i, doc in enumerate(retrieved_docs, 1):
            source = doc.metadata.get('source', 'Unknown')
            preview = doc.page_content[:200].replace('\n', ' ').strip()
            print(f"\n  [{i}] {source}")
            print(f"      Preview: {preview}...")
            
            if show_retrieved_docs:
                print(f"\n      Full content:\n{doc.page_content}\n")
    
    # Show timing
    if show_timing:
        print(f"\n⏱️  Response time: {elapsed_time:.2f} seconds")
    
    print("="*80 + "\n")
    
    return {
        "question": question,
        "answer": response,
        "time": elapsed_time
    }

print("✅ Advanced question function ready")

In [ ]:
def ask_quick(question: str):
    """Get a quick, concise answer"""
    quick_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a medical assistant. Answer briefly in 2-3 sentences.\n\nContext: {context}"),
        ("human", "{input}"),
    ])
    
    quick_chain = (
        {"context": retriever | format_docs, "input": RunnablePassthrough()}
        | quick_prompt
        | chatModel
        | StrOutputParser()
    )
    
    print(f"❓ {question}")
    response = quick_chain.invoke(question)
    print(f"💡 {response}\n")
    return response

In [ ]:
def ask_simple(question: str):
    """Get an easy-to-understand answer"""
    simple_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a caring doctor. Explain in simple language without medical jargon.\n\nContext: {context}"),
        ("human", "{input}"),
    ])
    
    simple_chain = (
        {"context": retriever | format_docs, "input": RunnablePassthrough()}
        | simple_prompt
        | chatModel
        | StrOutputParser()
    )
    
    print(f"❓ {question}")
    response = simple_chain.invoke(question)
    print(f"💡 {response}\n")
    return response

print("✅ Multiple question modes ready:")
print("   • ask_medical_question() - Detailed, comprehensive")
print("   • ask_quick()            - Brief, 2-3 sentences")
print("   • ask_simple()           - Easy language, no jargon")

In [ ]:
result = ask_medical_question(
    "What is acne? Explain its causes, symptoms, and treatment options in detail.",
    show_sources=True,
    show_timing=True
)

In [ ]:
test_questions = [
    "What is diabetes mellitus? Explain the types and management.",
    "What are the main symptoms and treatment of hypertension?",
    "Describe pneumonia, its causes, and how it's diagnosed."
]

print("\n" + "="*80)
print("🧪 TESTING MULTIPLE QUESTIONS")
print("="*80)

for i, q in enumerate(test_questions, 1):
    print(f"\n[Question {i}/{len(test_questions)}]\n")
    ask_medical_question(q, show_sources=False, show_timing=False)
    print("\n" + "-"*80)

In [ ]:
def interactive_chat():
    """Start interactive medical chatbot session"""
    
    print("\n" + "="*80)
    print("🏥 INTERACTIVE MEDICAL CHATBOT")
    print("="*80)
    print("\nAsk detailed medical questions. Type 'quit' to exit.\n")
    
    while True:
        question = input("❓ Your question: ").strip()
        
        if question.lower() in ['quit', 'exit', 'q', '']:
            print("\n👋 Session ended. Stay healthy!")
            break
        
        ask_medical_question(question, show_sources=True)

In [ ]:
def compare_answers(question: str):
    """Compare different answer modes side by side"""
    
    print("\n" + "="*80)
    print(f"📊 COMPARING ANSWER MODES")
    print(f"Question: {question}")
    print("="*80 + "\n")
    
    print("🔹 QUICK MODE (Concise):")
    print("-" * 80)
    ask_quick(question)
    
    print("\n🔹 SIMPLE MODE (Patient-friendly):")
    print("-" * 80)
    ask_simple(question)
    
    print("\n🔹 DETAILED MODE (Comprehensive):")
    print("-" * 80)
    ask_medical_question(question, show_sources=False, show_timing=False)

# Test comparison
compare_answers("What is asthma?")